# Data Cleaning
Import the data and clean for EDA. Drop columns that don't relate to our analysis, drop rows with unusable data or that are not in our time frame (2015-2019).

In [1]:
import pandas as pd
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')

__Read in Files__ from csv into pandas dataframes.

In [2]:
property_2019_full    = pd.read_csv('data/property-assessment-fy2019.csv')
property_2018_full    = pd.read_csv('data/property-assessment-fy2018.csv')
property_2017_full    = pd.read_csv('data/property-assessment-fy2017.csv')
property_2016_full    = pd.read_csv('data/property-assessment-fy2016.csv')
property_2015_full    = pd.read_csv('data/property-assessment-fy2015.csv')
streetlights_full     = pd.read_csv('data/streetlight_locations.csv')
crime_incidents_full  = pd.read_csv('data/crime_incident_reports.csv')
weather_full          = pd.read_csv('data/weather_boston.csv')

__Drop Columns__ after careful inspection of the data contained in each dataset, drop columns that will not help in our modeling. Columns were dropped if they had no effect on the outcome of interest (such as indeces or number of fireplaces in a property) or if the information in them was a duplicate (such as location if we were already given longitude and latitude).

1. from `streetlamps` drop everything but `Long` and `Lat` 
2. from `property_assessment` we only care where the property is and what it's valued at so drop everything that doesn't relate; `AV_TOTAL` is nonzero for most entries, and is the easiest representative of a property's value
3. from `crime_incidents` drop `Location` and the index, since the location information was duplicationg `Long` and `Lat` and the index was not useful for analysis

In [3]:
# drop everything but lat and long
streetlights = streetlights_full.drop(['the_geom','TYPE','OBJECTID'],axis=1)

In [4]:
# properties columns to save  
property_cols = ['ST_NUM','ST_NAME','ST_NAME_SUF','ZIPCODE','AV_TOTAL']

# drop all columns not in list (keep _ at end of name to show not fully clean yet)
property_2019_ = property_2019_full[property_2019_full.columns[property_2019_full.columns.isin(property_cols)]]
property_2018_ = property_2018_full[property_2018_full.columns[property_2018_full.columns.isin(property_cols)]]
property_2017_ = property_2017_full[property_2017_full.columns[property_2017_full.columns.isin(property_cols)]]
property_2016_ = property_2016_full[property_2016_full.columns[property_2016_full.columns.isin(property_cols)]]
property_2015_ = property_2015_full[property_2015_full.columns[property_2015_full.columns.isin(property_cols)]]

In [5]:
# crime columns to keep
crime_col = ['OCCURRED_ON_DATE','UCR_PART','HOUR','SHOOTING','DAY_OF_WEEK','Lat','Long']

# drop unwanted crime cols
crime_incidents_ = crime_incidents_full[crime_incidents_full.columns[crime_incidents_full.columns.isin(crime_col)]]

In [6]:
# weather columns to keep
weather_cols = ['DATE','PRCP','SNOW','TAVG']
weather_boston = weather_full[weather_full.columns[weather_full.columns.isin(weather_cols)]]

__Drop Rows__ that would not be usable in the forseeable future. This includes rows that have no predictor data, or no response variable data, in the form of 'nan' or 'none' or in some cases zeros. Careful inspection of each dataset led us to drop the following:
1. the `streetlights` dataset had no rows with immediately visible issues
2. from `property_assessment` we dropped all rows that had 0 in all four of the price variables, no issues with location were immediately visible
3. from `crime_incidents` we dropped if `Lat` and `Long` did not have usable values because it would be hard to get that information just from the street name and it is vital to our analysis

In [7]:
def property_droprows(df):
    
    # drop row if price is 0
    df_new = df[df.AV_TOTAL > 0]
    
    # drop rows with no street number because then the lat long is very off
    df_new = df_new[df_new['ST_NUM'].str.strip().astype(bool)]
    
    # get rid of pesky date number
    df_new = df_new[df_new.ST_NUM != '5-Feb']
    
    return(df_new)

In [8]:
# # drop property rows for all years
property_2019 = property_droprows(property_2019_)
property_2018 = property_droprows(property_2018_)
property_2017 = property_droprows(property_2017_)
property_2016 = property_droprows(property_2016_)
property_2015 = property_droprows(property_2015_)

In [9]:
# drop rows with nan long and lat 
crime_incidents = crime_incidents_.dropna(subset=['Lat','Long'])

# drop rows with 0 or -1 long and lat
crime_incidents = crime_incidents[crime_incidents.Lat != 0]
crime_incidents = crime_incidents[crime_incidents.Long != -1]

# tidy up shooting catigory to be binary
crime_incidents['SHOOTING'].fillna(0,inplace=True)
crime_incidents['SHOOTING'].replace('Y',1,inplace=True)

# drop any URC NaN because that's the response variable and theres only 109 of them
crime_incidents = crime_incidents[~crime_incidents.UCR_PART.isna()]

# drop URC that are Other, leaving only parts 1,2,3 and theres only 1,600 of them
crime_incidents = crime_incidents[~crime_incidents.UCR_PART.isin(['Other'])]

In [10]:
# get list of all days with NaN temp
nan_temps = list(weather_boston[weather_boston.TAVG.isna()].index)

# for rows with NaN average temp, replace with previous day's temp
for day in nan_temps:
    weather_boston.at[day,'TAVG'] = weather_boston.loc[day-1].TAVG

__Export to CSV__ export cleaned data to csv in the folder data for cleaner use.

In [12]:
property_2019.to_csv('data/property_2019.csv')
property_2018.to_csv('data/property_2018.csv')
property_2017.to_csv('data/property_2017.csv')
property_2016.to_csv('data/property_2016.csv')
property_2015.to_csv('data/property_2015.csv')

weather_boston.to_csv('data/weather_cleaned.csv')
crime_incidents.to_csv('data/crime_incidents.csv')
streetlights.to_csv('data/streetlights.csv')